In [ ]:
#import libraries
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time 

In [ ]:
#get URL and send GET request
url = "https://books.toscrape.com/"
response = requests.get(url)
if response.status_code == 200:
    print("request successful")
else:
    print("request failed")

In [ ]:
#parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")
print(soup)

In [ ]:
#extract book details for page 1
#find all the book titles and ther links
books = soup.find_all('h3')

start_time = time.time()
books_extracted = 0

#iterate through the books and extract the information for each book
for book in books:
    book_url = book.find('a')['href']
    book_response = requests.get(url + book_url)
    book_soup = BeautifulSoup(book_response.content, "html.parser")
    
    title = book_soup.find('h1').text
    category = book_soup.find('ul', class_="breadcrumb").find_all('a')[2].text.strip()
    rating = book_soup.find('p', class_='star-rating')['class'][1]
    price = book_soup.find('p', class_='price_color').text.strip()
    availability = book_soup.find('p', class_='availability').text.strip()
    
    books_extracted += 1
    
    end_time = time.time()
    total_time = (end_time-start_time)/60.00
    
    print(f'Title: {title}')
    print(f'Category: {category}')
    print(f'Rating: {rating}')
    print(f'Price: {price}')
    print(f'Availability: {availability}')
    print('*******')

In [ ]:
#extract details for all 50 pages
#create a list to hold all the book informaion
books_data = []

#loop trough all 50 pages
for page_num in range(1,51):
    url = f'https://books.toscrape.com/catalogue/page-{page_num}.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    books = soup.find_all('h3')

    start_time = time.time()
    books_extracted = 0

    for book in books:
        book_url = book.find('a')['href']
        book_response = requests.get('https://books.toscrape.com/catalogue/' + book_url)
        book_soup = BeautifulSoup(book_response.content, "html.parser")

        title = book_soup.find('h1').text
        category = book_soup.find('ul', class_="breadcrumb").find_all('a')[2].text.strip()
        rating = book_soup.find('p', class_='star-rating')['class'][1]
        price = book_soup.find('p', class_='price_color').text.strip()
        availability = book_soup.find('p', class_='availability').text.strip()

        end_time = time.time()
        total_time = (end_time-start_time)/60.00
        
        books_data.append([title, category, rating, price, availability])
        print(books_data)
        print('*******')
        print(f'{page_num * len(books)} books extracted so far...')


#add extracted info to the list

In [ ]:
#export the data
#convert list to dataframe
df = pd.DataFrame(books_data, columns=["Title", "Category", "Rating", "Price", "Availability"])
print(df.head(10))

In [ ]:
#save to csv
df.to_csv("books_scraped.csv", index=False)
print("Data saved to books_scraped.csv")